In [1]:
import sys
sys.path
sys.executable

'/usr/bin/python3'

In [2]:
import load_data
df = load_data.load_county_level(data_dir='data')

loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [2]:
for name in df.keys():
    print(name)

countyFIPS
STATEFP
COUNTYFP
CountyName
StateName
State
lat
lon
POP_LATITUDE
POP_LONGITUDE
CensusRegionName
CensusDivisionName
Rural-UrbanContinuumCode2013
PopulationEstimate2018
PopTotalMale2017
PopTotalFemale2017
FracMale2017
PopulationEstimate65+2017
PopulationDensityperSqMile2010
CensusPopulation2010
MedianAge2010
#EligibleforMedicare2018
MedicareEnrollment,AgedTot2017
3-YrDiabetes2015-17
DiabetesPercentage
HeartDiseaseMortality
StrokeMortality
Smokers_Percentage
RespMortalityRate2014
#FTEHospitalTotal2017
TotalM.D.'s,TotNon-FedandFed2017
#HospParticipatinginNetwork2017
#Hospitals
#ICU_beds
dem_to_rep_ratio
PopMale<52010
PopFmle<52010
PopMale5-92010
PopFmle5-92010
PopMale10-142010
PopFmle10-142010
PopMale15-192010
PopFmle15-192010
PopMale20-242010
PopFmle20-242010
PopMale25-292010
PopFmle25-292010
PopMale30-342010
PopFmle30-342010
PopMale35-442010
PopFmle35-442010
PopMale45-542010
PopFmle45-542010
PopMale55-592010
PopFmle55-592010
PopMale60-642010
PopFmle60-642010
PopMale65-742010
P

In [10]:
keys = df.keys()
for i, val in enumerate(df.iloc[0]):
    print(i, '\t', keys[i], '\t', val)

0 	 countyFIPS 	 36047
1 	 STATEFP 	 36.0
2 	 COUNTYFP 	 47.0
3 	 CountyName 	 Kings
4 	 StateName 	 NY
5 	 State 	 New York
6 	 lat 	 40.64057761
7 	 lon 	 -73.95465424
8 	 POP_LATITUDE 	 40.650523
9 	 POP_LONGITUDE 	 -73.95481099999998
10 	 CensusRegionName 	 Northeast
11 	 CensusDivisionName 	 Middle Atlantic
12 	 Rural-UrbanContinuumCode2013 	 1.0
13 	 PopulationEstimate2018 	 2582830.0
14 	 PopTotalMale2017 	 1254966.0
15 	 PopTotalFemale2017 	 1393805.0
16 	 FracMale2017 	 0.4737918075968062
17 	 PopulationEstimate65+2017 	 356714.0
18 	 PopulationDensityperSqMile2010 	 35369.2
19 	 CensusPopulation2010 	 2504700.0
20 	 MedianAge2010 	 34.1
21 	 #EligibleforMedicare2018 	 386673.0
22 	 MedicareEnrollment,AgedTot2017 	 309657.0
23 	 3-YrDiabetes2015-17 	 635.0
24 	 DiabetesPercentage 	 9.7
25 	 HeartDiseaseMortality 	 195.8
26 	 StrokeMortality 	 19.5
27 	 Smokers_Percentage 	 13.840801998
28 	 RespMortalityRate2014 	 23.05
29 	 #FTEHospitalTotal2017 	 35442.0
30 	 TotalM.D.'s,Tot

## Build Graphs

### Contiguous graph

Counties are connected if they are next to each other.

In [11]:
import numpy as np

county_names = df['CountyName']
county_code = np.array([int(code) for code in df['countyFIPS']])
county_code_dict = {code:idx for idx, code in enumerate(county_code)}

In [12]:
import pandas as pd
data_contiguous = pd.read_csv('/home/ubuntu/covid19-severity-prediction/data/neighborcounties.csv')
df_contiguous = pd.DataFrame(data_contiguous, columns= ['orgfips','adjfips','instate'])

In [13]:
import dgl

g = dgl.DGLGraph()
g.add_nodes(len(county_code_dict))
for ind in df_contiguous.index: 
    src = int(df_contiguous['orgfips'][ind])
    dst = int(df_contiguous['adjfips'][ind])
    if src in county_code_dict and dst in county_code_dict:
        g.add_edge(county_code_dict[src],county_code_dict[dst])

### Load attributes of counties

In [14]:
import numpy as np
import torch as th
attr_cols = []

# Set the variable to True if we want to specify features ourselves.
use_specified_fields = False

if use_specified_fields:
    f = open("fields.txt", "r")
    fields = f.readlines()
    for field in fields:
        field = field.rstrip()
        arr = np.array(df[field])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))
else:
    for i in range(6, 86):
        arr = np.array(df[keys[i]])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))

attrs = th.tensor(np.stack(attr_cols, 1), dtype=th.float32)
cases = []
for i in range(87, 216):
    cases.append(np.array(df[keys[i]]))
cases = np.stack(cases, 1)
deaths = []
for i in range(218, 347):
    deaths.append(np.array(df[keys[i]]))
deaths = np.stack(deaths, 1)

pop = attrs[:,5]
pop = pop.view(3142, 1)
pop[1834] = 96471 #fixed the population value for Erie

print('attrs:', attrs.shape)
print('cases:', cases.shape)
print('deaths:', deaths.shape)
print(attrs[0])
print(cases[0])
print(deaths[0])

attrs: torch.Size([3142, 78])
cases: (3142, 129)
deaths: (3142, 129)
tensor([ 4.0641e+01, -7.3955e+01,  4.0651e+01, -7.3955e+01,  1.0000e+00,
         2.5828e+06,  1.2550e+06,  1.3938e+06,  4.7379e-01,  3.5671e+05,
         3.5369e+04,  2.5047e+06,  3.4100e+01,  3.8667e+05,  3.0966e+05,
         6.3500e+02,  9.7000e+00,  1.9580e+02,  1.9500e+01,  1.3841e+01,
         2.3050e+01,  3.5442e+04,  9.3070e+03,  6.0000e+00,  1.2000e+01,
         3.1800e+02,  4.5415e+00,  9.0415e+04,  8.6783e+04,  8.1356e+04,
         7.8035e+04,  7.9604e+04,  7.6959e+04,  8.6716e+04,  8.3968e+04,
         9.6663e+04,  9.9134e+04,  1.0605e+05,  1.1679e+05,  9.7154e+04,
         1.0570e+05,  1.6196e+05,  1.7958e+05,  1.4949e+05,  1.7469e+05,
         6.3789e+04,  7.8116e+04,  5.5241e+04,  6.8866e+04,  6.4628e+04,
         8.8066e+04,  3.5780e+04,  5.8209e+04,  1.2524e+04,  2.8426e+04,
         1.5400e+02,  2.4000e+01,  3.3000e+01,  1.5100e+02,  3.4100e+02,
         4.9300e+02,  1.1160e+03,  2.1390e+03,  2.8520e

## Save SIR data to csv file

In [16]:
# "\\home\\ubuntu\\covid19-severity-prediction\\county_csv\\0_Autauga_36047.csv"
pop = attrs[:,5]
import pandas as pd
path = '/home/ubuntu/covid19-severity-prediction/county_csv/'
for idx, code in enumerate(county_code):
    csv_data = {'Days': range(129),
                'S': [pop[idx].item() - cases[idx] - deaths[idx] for i in range(129)],
                'I': cases[idx],
                'D': deaths[idx]}
    csv_df = pd.DataFrame(csv_data, columns = ['Days', 'S', 'I', 'R'])
    csv_name = path + str(idx)+'_' + str(county_names[idx])+ '_' + str(code) +'.csv'
    csv_df.to_csv(csv_name)

In [29]:
idx = 1834
csv_data = {'Days': range(129),
                'S': [pop[idx].item() - cases[idx][k] - deaths[idx][k] for k in range(129)],
                'I': cases[idx],
                'D': deaths[idx]}


'Erie'

## Loss 
### Mean Square Log Error

In [8]:
def my_msle(output, target):
    loss = th.mean(((output.add(1)).log() - (target.add(1)).log())**2)
    return loss

In [9]:
"""
Input: beta(from NN), gamma(from NN), N(from attributes)
Output: mean square log error for infected cases
"""
def sir_loss(params, I, D, target):
    I[I != I] = 0.0
    D[D != D] = 0.0
    target[target != target] = 0.0
    err = th.tensor([0],dtype=th.float)
    err = err.to(device)
    S = pop - I - D
    l = int(len(params[0]) / 2)
    for i in range(l):
        # print("=============================" + str(i) + "=============================")
        S_new = th.add(S, - th.div(th.mul(params[:,i + l].view(3142,1), th.mul(S, I)), pop))
        I_new = th.add(th.add(I, th.mul(params[:,i + l].view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(params[:,i].view(3142,1), I))
        D_new = th.add(D, th.mul(params[:,i].view(3142,1), I))
        temp = (I_new.add(1).log() - target[:,i].view(3142,1).add(1).log()).pow(2)
        # m_temp = th.mean(temp)
        # print(m_temp)
        # if m_temp!= m_temp:
        #     for j,k in enumerate(temp):
        #         print(j,k)
        err = err + th.mean(temp)
        S = S_new
        I = I_new
        D = D_new
    # print(err)
    return err

In [10]:
def sir_loss_const(params, I, D, target):
    gamma = params[:,0]
    beta = params[:,1]
    I[I != I] = 0.0
    D[D != D] = 0.0
    target[target != target] = 0.0
    err = th.tensor([0],dtype=th.float)
    err = err.to(device)
    S = pop - I - D
    for i in range(7):
        S_new = th.add(S, - th.div(th.mul(beta.view(3142,1), th.mul(S, I)), pop))
        I_new = th.add(th.add(I, th.mul(beta.view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(gamma.view(3142,1), I))
        D_new = th.add(D, th.mul(gamma.view(3142,1), I))
        temp = (I_new.add(1).log() - target[:,i].view(3142,1).add(1).log()).pow(2)
        err = err + th.mean(temp)
        S = S_new
        I = I_new
        D = D_new
    return err

## Prepare data

In [11]:
import torch as th

out_size = 1
num_days = cases.shape[1]
num_valid_days = 15
num_test_days = 15
train_valid_split = num_days - num_valid_days - num_test_days
valid_test_split = num_days - num_test_days
# number of days for input
in_len = 20
# number of days for output
out_len = 1

train_past_cases = []
train_past_deaths = []
train_labels_cases = []
train_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    train_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    train_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    train_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    train_labels_deaths.append(th.tensor(deaths[:,end_begin:end], dtype=th.float32))

valid_past_cases = []
valid_past_deaths = []
valid_labels_cases = []
valid_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1, valid_test_split - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    valid_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    valid_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    valid_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    valid_labels_deaths.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    
test_past_cases = []
test_past_deaths = []
test_labels_cases = []
test_labels_deaths = []
for start in range(valid_test_split - in_len - out_len + 1, num_days - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    test_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    test_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    test_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    test_labels_deaths.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))

## MLP model¶

In [12]:
def init_weights(m):
    if type(m) == nn.Linear:
        th.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [13]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.nn_model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            #nn.BatchNorm1d(num_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.nn_model(x)

In [14]:
import torch.nn as nn

class Net_SIR(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net_SIR, self).__init__()
        self.nn_model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            #nn.BatchNorm1d(num_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.nn_model(x)

### MLP training loop

In [191]:
hidden_size = 50
out_size = 1
learning_rate = 0.000001
num_epochs = 1000
batch_size = 1000

# print('mean of training labels: {:.3f}, mean of testing labels: {:.3f}'.format(th.mean(train_labels_cases),
#                                                                                th.mean(valid_labels_cases)))

device = th.device('cuda:0')
input_size = train_past_cases[0].shape[1] + attrs.shape[1]
net_mlp = Net(input_size, hidden_size, out_size)
# net.apply(init_weights)
# criterion = nn.MSELoss()
print('#features:', input_size)

# Move model to GPU
net_mlp = net_mlp.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
        
for i in range(len(valid_past_cases)):
    valid_past_cases[i] = valid_past_cases[i].to(device)
    valid_labels_cases[i] = valid_labels_cases[i].to(device)
    if len(valid_labels_cases[i].shape) == 1:
         valid_labels_cases[i] = valid_labels_cases[i].unsqueeze(1)

for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1) 

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_mlp.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    sample_idxs = th.tensor(range(len(train_past_cases)))
    batch_idxs = th.split(sample_idxs, batch_size)
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        #batch = train_past_cases[idx]
        vals = net_mlp(batch)
        # loss = criterion(vals, labels)
        loss = my_msle(vals, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())

        
    with th.no_grad():
        eval_errs = []
        for cases, labels in zip(valid_past_cases, valid_labels_cases):
            valid_feats = th.cat([cases, attrs], dim=1)
            eval_vals = net_mlp(valid_feats)
            err = my_msle(eval_vals, labels)
            eval_errs.append(err.cpu().numpy())
        
        test_errs = []
        test_mses = []
        test_maes = []
        for idx in range(6):
            test_feats = th.cat([test_past_cases[idx], attrs], dim=1)
            test_vals = net_mlp(test_feats)
            test_labels = test_labels_cases[idx]
            test_err = my_msle(test_vals, test_labels)
            test_errs.append(test_err.cpu().numpy())
            test_mse = th.mean((test_vals - test_labels)**2)
            test_mses.append(test_mse.cpu().numpy())
            test_mae = th.mean(th.abs(test_vals - test_labels))
            test_maes.append(test_mae.cpu().numpy())
    print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs), np.mean(test_mses), np.mean(test_maes)))

#features: 97
epoch=0, loss=2.741, validation loss = 8.713, test loss=9.344, test mse = 7571657.500, test mae = 458.006
epoch=1, loss=2.684, validation loss = 8.456, test loss=9.048, test mse = 7548649.500, test mae = 457.341
epoch=2, loss=2.631, validation loss = 8.216, test loss=8.773, test mse = 7525504.500, test mae = 456.671
epoch=3, loss=2.581, validation loss = 7.997, test loss=8.523, test mse = 7502849.500, test mae = 456.013
epoch=4, loss=2.535, validation loss = 7.795, test loss=8.293, test mse = 7480580.500, test mae = 455.363
epoch=5, loss=2.491, validation loss = 7.605, test loss=8.079, test mse = 7458302.000, test mae = 454.712
epoch=6, loss=2.449, validation loss = 7.428, test loss=7.879, test mse = 7436254.000, test mae = 454.064
epoch=7, loss=2.409, validation loss = 7.260, test loss=7.691, test mse = 7414286.000, test mae = 453.418
epoch=8, loss=2.372, validation loss = 7.102, test loss=7.515, test mse = 7392216.500, test mae = 452.772
epoch=9, loss=2.337, validation 

epoch=77, loss=1.224, validation loss = 2.663, test loss=2.659, test mse = 5699613.500, test mae = 399.778
epoch=78, loss=1.214, validation loss = 2.627, test loss=2.621, test mse = 5668794.500, test mae = 398.751
epoch=79, loss=1.204, validation loss = 2.592, test loss=2.583, test mse = 5637830.500, test mae = 397.716
epoch=80, loss=1.194, validation loss = 2.557, test loss=2.546, test mse = 5606740.000, test mae = 396.672
epoch=81, loss=1.184, validation loss = 2.523, test loss=2.510, test mse = 5575541.500, test mae = 395.621
epoch=82, loss=1.174, validation loss = 2.488, test loss=2.474, test mse = 5544173.500, test mae = 394.560
epoch=83, loss=1.164, validation loss = 2.455, test loss=2.438, test mse = 5512188.500, test mae = 393.477
epoch=84, loss=1.154, validation loss = 2.421, test loss=2.403, test mse = 5479535.500, test mae = 392.371
epoch=85, loss=1.144, validation loss = 2.388, test loss=2.367, test mse = 5446542.500, test mae = 391.250
epoch=86, loss=1.135, validation loss

epoch=155, loss=0.638, validation loss = 0.763, test loss=0.688, test mse = 2687991.500, test mae = 280.230
epoch=156, loss=0.633, validation loss = 0.749, test loss=0.673, test mse = 2644731.250, test mae = 278.104
epoch=157, loss=0.628, validation loss = 0.734, test loss=0.659, test mse = 2601555.750, test mae = 275.965
epoch=158, loss=0.622, validation loss = 0.720, test loss=0.645, test mse = 2558458.000, test mae = 273.812
epoch=159, loss=0.617, validation loss = 0.706, test loss=0.631, test mse = 2515447.750, test mae = 271.644
epoch=160, loss=0.612, validation loss = 0.692, test loss=0.617, test mse = 2472540.250, test mae = 269.462
epoch=161, loss=0.607, validation loss = 0.679, test loss=0.604, test mse = 2429738.750, test mae = 267.265
epoch=162, loss=0.601, validation loss = 0.665, test loss=0.590, test mse = 2387049.500, test mae = 265.053
epoch=163, loss=0.596, validation loss = 0.652, test loss=0.577, test mse = 2344471.000, test mae = 262.825
epoch=164, loss=0.591, valid

epoch=233, loss=0.301, validation loss = 0.120, test loss=0.078, test mse = 113393.070, test mae = 60.430
epoch=234, loss=0.298, validation loss = 0.117, test loss=0.075, test mse = 103417.992, test mae = 56.997
epoch=235, loss=0.295, validation loss = 0.115, test loss=0.073, test mse = 94411.812, test mae = 53.748
epoch=236, loss=0.292, validation loss = 0.112, test loss=0.071, test mse = 86391.406, test mae = 50.852
epoch=237, loss=0.289, validation loss = 0.109, test loss=0.069, test mse = 79373.758, test mae = 48.340
epoch=238, loss=0.285, validation loss = 0.107, test loss=0.068, test mse = 73376.086, test mae = 46.369
epoch=239, loss=0.282, validation loss = 0.104, test loss=0.066, test mse = 68415.914, test mae = 45.088
epoch=240, loss=0.279, validation loss = 0.102, test loss=0.064, test mse = 64510.426, test mae = 44.266
epoch=241, loss=0.276, validation loss = 0.100, test loss=0.063, test mse = 61676.879, test mae = 43.619
epoch=242, loss=0.272, validation loss = 0.097, test 

epoch=310, loss=0.142, validation loss = 0.162, test loss=0.191, test mse = 3846407.250, test mae = 286.218
epoch=311, loss=0.141, validation loss = 0.166, test loss=0.196, test mse = 3980452.750, test mae = 291.562
epoch=312, loss=0.141, validation loss = 0.170, test loss=0.201, test mse = 4117454.750, test mae = 296.932
epoch=313, loss=0.140, validation loss = 0.173, test loss=0.206, test mse = 4257448.500, test mae = 302.332
epoch=314, loss=0.139, validation loss = 0.177, test loss=0.211, test mse = 4400462.500, test mae = 307.759
epoch=315, loss=0.138, validation loss = 0.181, test loss=0.216, test mse = 4546531.500, test mae = 313.214
epoch=316, loss=0.138, validation loss = 0.185, test loss=0.221, test mse = 4695672.500, test mae = 318.696
epoch=317, loss=0.137, validation loss = 0.189, test loss=0.226, test mse = 4847916.000, test mae = 324.205
epoch=318, loss=0.136, validation loss = 0.194, test loss=0.231, test mse = 5003298.500, test mae = 329.741
epoch=319, loss=0.136, valid

...

epoch=686, loss=0.371, validation loss = 1.986, test loss=2.282, test mse = 210507840.000, test mae = 2276.662
epoch=687, loss=0.370, validation loss = 1.985, test loss=2.281, test mse = 210578768.000, test mae = 2277.070
epoch=688, loss=0.370, validation loss = 1.985, test loss=2.281, test mse = 210641168.000, test mae = 2277.432
epoch=689, loss=0.370, validation loss = 1.985, test loss=2.281, test mse = 210695120.000, test mae = 2277.748
epoch=690, loss=0.370, validation loss = 1.984, test loss=2.280, test mse = 210740528.000, test mae = 2278.018
epoch=691, loss=0.370, validation loss = 1.984, test loss=2.280, test mse = 210777584.000, test mae = 2278.242
epoch=692, loss=0.370, validation loss = 1.983, test loss=2.279, test mse = 210805968.000, test mae = 2278.420
epoch=693, loss=0.370, validation loss = 1.983, test loss=2.279, test mse = 210825936.000, test mae = 2278.552
epoch=694, loss=0.370, validation loss = 1.982, test loss=2.278, test mse = 210837440.000, test mae = 2278.638
e

epoch=760, loss=0.344, validation loss = 1.846, test loss=2.134, test mse = 193091024.000, test mae = 2181.620
epoch=761, loss=0.343, validation loss = 1.843, test loss=2.130, test mse = 192542640.000, test mae = 2178.524
epoch=762, loss=0.342, validation loss = 1.840, test loss=2.126, test mse = 191985264.000, test mae = 2175.372
epoch=763, loss=0.342, validation loss = 1.836, test loss=2.122, test mse = 191418928.000, test mae = 2172.164
epoch=764, loss=0.341, validation loss = 1.832, test loss=2.118, test mse = 190843856.000, test mae = 2168.901
epoch=765, loss=0.340, validation loss = 1.829, test loss=2.114, test mse = 190259984.000, test mae = 2165.582
epoch=766, loss=0.339, validation loss = 1.825, test loss=2.110, test mse = 189667248.000, test mae = 2162.208
epoch=767, loss=0.339, validation loss = 1.821, test loss=2.106, test mse = 189065968.000, test mae = 2158.779
epoch=768, loss=0.338, validation loss = 1.817, test loss=2.102, test mse = 188456144.000, test mae = 2155.295
e

epoch=834, loss=0.271, validation loss = 1.461, test loss=1.707, test mse = 132424896.000, test mae = 1807.510
epoch=835, loss=0.270, validation loss = 1.454, test loss=1.699, test mse = 131376472.000, test mae = 1800.310
epoch=836, loss=0.268, validation loss = 1.447, test loss=1.690, test mse = 130327464.000, test mae = 1793.077
epoch=837, loss=0.267, validation loss = 1.439, test loss=1.682, test mse = 129277768.000, test mae = 1785.810
epoch=838, loss=0.266, validation loss = 1.432, test loss=1.674, test mse = 128227232.000, test mae = 1778.510
epoch=839, loss=0.264, validation loss = 1.424, test loss=1.665, test mse = 127176024.000, test mae = 1771.175
epoch=840, loss=0.263, validation loss = 1.417, test loss=1.657, test mse = 126124376.000, test mae = 1763.807
epoch=841, loss=0.262, validation loss = 1.409, test loss=1.649, test mse = 125072504.000, test mae = 1756.405
epoch=842, loss=0.260, validation loss = 1.402, test loss=1.640, test mse = 124020256.000, test mae = 1748.967
e

epoch=910, loss=0.161, validation loss = 0.800, test loss=0.959, test mse = 54365088.000, test mae = 1154.372
epoch=911, loss=0.160, validation loss = 0.790, test loss=0.948, test mse = 53432176.000, test mae = 1144.322
epoch=912, loss=0.158, validation loss = 0.780, test loss=0.937, test mse = 52504372.000, test mae = 1134.239
epoch=913, loss=0.157, validation loss = 0.771, test loss=0.925, test mse = 51581872.000, test mae = 1124.125
epoch=914, loss=0.156, validation loss = 0.761, test loss=0.914, test mse = 50664768.000, test mae = 1113.979
epoch=915, loss=0.154, validation loss = 0.751, test loss=0.903, test mse = 49753156.000, test mae = 1103.803
epoch=916, loss=0.153, validation loss = 0.741, test loss=0.891, test mse = 48847136.000, test mae = 1093.596
epoch=917, loss=0.151, validation loss = 0.731, test loss=0.880, test mse = 47946848.000, test mae = 1083.360
epoch=918, loss=0.150, validation loss = 0.722, test loss=0.868, test mse = 47052460.000, test mae = 1073.094
epoch=919,

epoch=986, loss=0.113, validation loss = 0.237, test loss=0.261, test mse = 5912184.000, test mae = 373.879
epoch=987, loss=0.114, validation loss = 0.236, test loss=0.258, test mse = 5631776.500, test mae = 364.687
epoch=988, loss=0.115, validation loss = 0.235, test loss=0.255, test mse = 5359710.000, test mae = 355.550
epoch=989, loss=0.116, validation loss = 0.234, test loss=0.252, test mse = 5095986.000, test mae = 346.471
epoch=990, loss=0.116, validation loss = 0.233, test loss=0.250, test mse = 4840549.500, test mae = 337.453
epoch=991, loss=0.117, validation loss = 0.233, test loss=0.248, test mse = 4593387.500, test mae = 328.500
epoch=992, loss=0.119, validation loss = 0.233, test loss=0.246, test mse = 4354472.500, test mae = 319.614
epoch=993, loss=0.120, validation loss = 0.234, test loss=0.245, test mse = 4123789.750, test mae = 310.802
epoch=994, loss=0.121, validation loss = 0.235, test loss=0.244, test mse = 3901278.750, test mae = 302.067
epoch=995, loss=0.122, valid

## GNN

### GAT - MSLE

In [15]:
from dgl.nn import pytorch as dglnn
import torch.nn.functional as F
import torch.nn as nn

class GAT(nn.Module):
    def __init__(self, g, input_size, hidden_size, output_size, gcn_nlayers, num_heads=5):
        super(GAT, self).__init__()
        self.g = g
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(dglnn.conv.GATConv(input_size, hidden_size, num_heads=num_heads,
                                                  residual=True, activation=F.relu))
        for i in range(gcn_nlayers):
            self.gcn_layers.append(dglnn.conv.GATConv(num_heads * hidden_size, hidden_size, num_heads=num_heads,
                                                      residual=True, activation=F.relu))

        self.linear_layers = nn.Sequential(
            nn.Linear(num_heads * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU()
        )

    def forward(self, x):
        h = x
        for layer in self.gcn_layers:
            h = layer(self.g, h).flatten(1)
        return self.linear_layers(h)

In [150]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 1000
batch_size = 1000
output_size = 1

device = th.device('cuda:0')
net_gat = GAT(g, train_past_cases[0].shape[1] + attrs.shape[1], hidden_size, output_size, 1)

# Move model to GPU
net_gat = net_gat.to(device)


# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
        
for i in range(len(valid_past_cases)):
    valid_past_cases[i] = valid_past_cases[i].to(device)
    valid_labels_cases[i] = valid_labels_cases[i].to(device)
    if len(valid_labels_cases[i].shape) == 1:
         valid_labels_cases[i] = valid_labels_cases[i].unsqueeze(1)

for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)            
            
# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_gat.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        #batch = train_past_cases[idx]
        vals = net_gat(batch)
        # loss = criterion(vals, labels)
        loss = my_msle(vals, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        eval_errs = []
        for cases, labels in zip(valid_past_cases, valid_labels_cases):
            valid_feats = th.cat([cases, attrs], dim=1)
            eval_vals = net_gat(valid_feats)
            err = my_msle(eval_vals, labels)
            eval_errs.append(err.cpu().numpy())
        
        test_errs = []
        test_mses = []
        test_maes = []
        for idx in range(6):
            test_feats = th.cat([test_past_cases[idx], attrs], dim=1)
            test_vals = net_gat(test_feats)
            test_labels = test_labels_cases[idx]
            test_err = my_msle(test_vals, test_labels)
            test_errs.append(test_err.cpu().numpy())
            test_mse = th.mean((test_vals - test_labels)**2)
            test_mses.append(test_mse.cpu().numpy())
            test_mae = th.mean(th.abs(test_vals - test_labels))
            test_maes.append(test_mae.cpu().numpy())
        print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs), np.mean(test_mses), np.mean(test_maes)))
        # print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs),np.mean(test_mses),np.mean(test_maes)))

epoch=0, loss=3.785, validation loss = 13.044, test loss=14.433, test mse = 7713035.500, test mae = 461.815
epoch=1, loss=2.921, validation loss = 8.458, test loss=9.430, test mse = 7581288.500, test mae = 453.993
epoch=2, loss=2.075, validation loss = 4.717, test loss=4.959, test mse = 7368371.500, test mae = 442.031
epoch=3, loss=1.652, validation loss = 3.461, test loss=3.485, test mse = 7181037.500, test mae = 432.010
epoch=4, loss=1.411, validation loss = 2.943, test loss=2.945, test mse = 6996453.500, test mae = 424.006
epoch=5, loss=1.255, validation loss = 2.654, test loss=2.665, test mse = 6849478.000, test mae = 418.135
epoch=6, loss=1.145, validation loss = 2.473, test loss=2.493, test mse = 6728265.500, test mae = 413.510
epoch=7, loss=1.069, validation loss = 2.349, test loss=2.379, test mse = 6607902.500, test mae = 409.463
epoch=8, loss=1.013, validation loss = 2.266, test loss=2.306, test mse = 6503560.500, test mae = 406.081
epoch=9, loss=0.973, validation loss = 2.210

epoch=77, loss=0.583, validation loss = 1.896, test loss=2.094, test mse = 2073929.375, test mae = 286.081
epoch=78, loss=0.577, validation loss = 1.874, test loss=2.069, test mse = 2045923.500, test mae = 283.795
epoch=79, loss=0.571, validation loss = 1.851, test loss=2.044, test mse = 2019301.625, test mae = 281.542
epoch=80, loss=0.565, validation loss = 1.828, test loss=2.019, test mse = 1993903.500, test mae = 279.314
epoch=81, loss=0.558, validation loss = 1.804, test loss=1.993, test mse = 1969315.125, test mae = 277.060
epoch=82, loss=0.552, validation loss = 1.780, test loss=1.966, test mse = 1945678.375, test mae = 274.911
epoch=83, loss=0.545, validation loss = 1.756, test loss=1.940, test mse = 1922944.625, test mae = 272.783
epoch=84, loss=0.539, validation loss = 1.731, test loss=1.913, test mse = 1901056.500, test mae = 270.745
epoch=85, loss=0.532, validation loss = 1.707, test loss=1.886, test mse = 1880095.625, test mae = 268.773
epoch=86, loss=0.526, validation loss

epoch=154, loss=0.234, validation loss = 0.599, test loss=0.648, test mse = 118018.633, test mae = 90.101
epoch=155, loss=0.229, validation loss = 0.583, test loss=0.628, test mse = 102081.586, test mae = 85.245
epoch=156, loss=0.225, validation loss = 0.569, test loss=0.611, test mse = 88501.258, test mae = 80.360
epoch=157, loss=0.221, validation loss = 0.557, test loss=0.596, test mse = 77508.281, test mae = 75.749
epoch=158, loss=0.217, validation loss = 0.546, test loss=0.584, test mse = 69104.734, test mae = 72.303
epoch=159, loss=0.214, validation loss = 0.536, test loss=0.573, test mse = 63301.984, test mae = 70.220
epoch=160, loss=0.211, validation loss = 0.528, test loss=0.564, test mse = 60072.168, test mae = 68.682
epoch=161, loss=0.207, validation loss = 0.520, test loss=0.556, test mse = 59409.062, test mae = 67.442
epoch=162, loss=0.204, validation loss = 0.513, test loss=0.549, test mse = 61309.262, test mae = 66.485
epoch=163, loss=0.201, validation loss = 0.506, test 

epoch=231, loss=0.109, validation loss = 0.399, test loss=0.475, test mse = 7575556.500, test mae = 383.556
epoch=232, loss=0.108, validation loss = 0.396, test loss=0.472, test mse = 7761392.500, test mae = 387.974
epoch=233, loss=0.107, validation loss = 0.392, test loss=0.468, test mse = 7937708.500, test mae = 391.881
epoch=234, loss=0.107, validation loss = 0.388, test loss=0.463, test mse = 8107468.000, test mae = 395.586
epoch=235, loss=0.106, validation loss = 0.382, test loss=0.459, test mse = 8271673.500, test mae = 398.994
epoch=236, loss=0.105, validation loss = 0.377, test loss=0.452, test mse = 8432139.000, test mae = 402.316
epoch=237, loss=0.104, validation loss = 0.371, test loss=0.446, test mse = 8586596.000, test mae = 405.272
epoch=238, loss=0.104, validation loss = 0.364, test loss=0.441, test mse = 8737559.000, test mae = 408.161
epoch=239, loss=0.103, validation loss = 0.362, test loss=0.440, test mse = 8892869.000, test mae = 411.162
epoch=240, loss=0.102, valid

epoch=307, loss=0.077, validation loss = 0.235, test loss=0.273, test mse = 12378365.000, test mae = 469.886
epoch=308, loss=0.077, validation loss = 0.238, test loss=0.275, test mse = 12359059.000, test mae = 469.488
epoch=309, loss=0.077, validation loss = 0.240, test loss=0.277, test mse = 12335031.000, test mae = 468.992
epoch=310, loss=0.077, validation loss = 0.242, test loss=0.279, test mse = 12306921.000, test mae = 468.401
epoch=311, loss=0.077, validation loss = 0.244, test loss=0.281, test mse = 12266104.000, test mae = 467.499
epoch=312, loss=0.077, validation loss = 0.245, test loss=0.284, test mse = 12219612.000, test mae = 466.458
epoch=313, loss=0.077, validation loss = 0.247, test loss=0.290, test mse = 12178037.000, test mae = 465.558
epoch=314, loss=0.077, validation loss = 0.248, test loss=0.292, test mse = 12134288.000, test mae = 464.617
epoch=315, loss=0.077, validation loss = 0.249, test loss=0.296, test mse = 12090669.000, test mae = 463.657
epoch=316, loss=0.0

...

epoch=616, loss=0.032, validation loss = 0.071, test loss=0.089, test mse = 52297.535, test mae = 31.985
epoch=617, loss=0.029, validation loss = 0.054, test loss=0.066, test mse = 56751.141, test mae = 33.606
epoch=618, loss=0.029, validation loss = 0.057, test loss=0.068, test mse = 64605.730, test mae = 37.581
epoch=619, loss=0.031, validation loss = 0.070, test loss=0.084, test mse = 70189.773, test mae = 40.405
epoch=620, loss=0.031, validation loss = 0.068, test loss=0.081, test mse = 69261.383, test mae = 39.940
epoch=621, loss=0.029, validation loss = 0.053, test loss=0.064, test mse = 63103.082, test mae = 36.538
epoch=622, loss=0.029, validation loss = 0.057, test loss=0.070, test mse = 57332.402, test mae = 33.455
epoch=623, loss=0.032, validation loss = 0.075, test loss=0.090, test mse = 55807.145, test mae = 33.129
epoch=624, loss=0.030, validation loss = 0.058, test loss=0.071, test mse = 58293.266, test mae = 33.735
epoch=625, loss=0.029, validation loss = 0.055, test lo

epoch=694, loss=0.030, validation loss = 0.063, test loss=0.079, test mse = 134414.375, test mae = 49.852
epoch=695, loss=0.030, validation loss = 0.072, test loss=0.073, test mse = 159697.125, test mae = 54.794
epoch=696, loss=0.042, validation loss = 0.144, test loss=0.159, test mse = 182440.328, test mae = 60.983
epoch=697, loss=0.038, validation loss = 0.101, test loss=0.106, test mse = 174687.766, test mae = 58.226
epoch=698, loss=0.028, validation loss = 0.052, test loss=0.068, test mse = 146345.500, test mae = 51.368
epoch=699, loss=0.033, validation loss = 0.102, test loss=0.130, test mse = 129758.648, test mae = 50.865
epoch=700, loss=0.042, validation loss = 0.144, test loss=0.183, test mse = 124870.352, test mae = 53.107
epoch=701, loss=0.045, validation loss = 0.151, test loss=0.191, test mse = 124483.727, test mae = 53.501
epoch=702, loss=0.039, validation loss = 0.117, test loss=0.149, test mse = 127997.000, test mae = 51.620
epoch=703, loss=0.030, validation loss = 0.064

epoch=772, loss=0.028, validation loss = 0.062, test loss=0.074, test mse = 123201.938, test mae = 50.420
epoch=773, loss=0.045, validation loss = 0.144, test loss=0.147, test mse = 172794.094, test mae = 62.626
epoch=774, loss=0.062, validation loss = 0.212, test loss=0.229, test mse = 188653.000, test mae = 67.102
epoch=775, loss=0.041, validation loss = 0.086, test loss=0.086, test mse = 146923.562, test mae = 55.781
epoch=776, loss=0.030, validation loss = 0.085, test loss=0.109, test mse = 97874.500, test mae = 46.633
epoch=777, loss=0.040, validation loss = 0.161, test loss=0.209, test mse = 76493.836, test mae = 46.996
epoch=778, loss=0.054, validation loss = 0.227, test loss=0.288, test mse = 74073.445, test mae = 50.212
epoch=779, loss=0.062, validation loss = 0.251, test loss=0.317, test mse = 75402.086, test mae = 51.937
epoch=780, loss=0.058, validation loss = 0.224, test loss=0.285, test mse = 73294.359, test mae = 50.016
epoch=781, loss=0.044, validation loss = 0.157, tes

epoch=850, loss=0.065, validation loss = 0.219, test loss=0.244, test mse = 100131.977, test mae = 49.807
epoch=851, loss=0.035, validation loss = 0.065, test loss=0.069, test mse = 57429.141, test mae = 35.166
epoch=852, loss=0.034, validation loss = 0.122, test loss=0.155, test mse = 40788.129, test mae = 36.718
epoch=853, loss=0.050, validation loss = 0.217, test loss=0.268, test mse = 54203.645, test mae = 45.422
epoch=854, loss=0.067, validation loss = 0.286, test loss=0.349, test mse = 75604.938, test mae = 54.059
epoch=855, loss=0.074, validation loss = 0.305, test loss=0.370, test mse = 83360.117, test mae = 57.051
epoch=856, loss=0.067, validation loss = 0.265, test loss=0.324, test mse = 69597.156, test mae = 51.507
epoch=857, loss=0.049, validation loss = 0.183, test loss=0.228, test mse = 47307.188, test mae = 41.907
epoch=858, loss=0.034, validation loss = 0.097, test loss=0.123, test mse = 40391.332, test mae = 34.517
epoch=859, loss=0.035, validation loss = 0.096, test l

epoch=928, loss=0.066, validation loss = 0.268, test loss=0.325, test mse = 80022.133, test mae = 56.861
epoch=929, loss=0.044, validation loss = 0.158, test loss=0.196, test mse = 38038.277, test mae = 37.581
epoch=930, loss=0.031, validation loss = 0.067, test loss=0.078, test mse = 27523.650, test mae = 28.825
epoch=931, loss=0.047, validation loss = 0.173, test loss=0.195, test mse = 57253.145, test mae = 38.445
epoch=932, loss=0.086, validation loss = 0.388, test loss=0.467, test mse = 93306.781, test mae = 52.496
epoch=933, loss=0.081, validation loss = 0.305, test loss=0.382, test mse = 79913.812, test mae = 47.574
epoch=934, loss=0.044, validation loss = 0.085, test loss=0.092, test mse = 39279.121, test mae = 31.331
epoch=935, loss=0.034, validation loss = 0.103, test loss=0.127, test mse = 27519.490, test mae = 31.082
epoch=936, loss=0.048, validation loss = 0.211, test loss=0.258, test mse = 56170.488, test mae = 46.400
epoch=937, loss=0.068, validation loss = 0.307, test lo

### GAT - SIR

In [16]:
device = th.device('cuda:0')
pop = attrs[:,5]
pop = pop.view(3142, 1)
pop[1834] = 1 # pop[pop==0] = 1
pop.to(device)

tensor([[2.5828e+06],
        [2.2789e+06],
        [1.4321e+06],
        ...,
        [2.7072e+04],
        [9.2530e+03],
        [8.8000e+01]], device='cuda:0')

In [17]:
from dgl.nn import pytorch as dglnn
import torch.nn.functional as F
import torch.nn as nn

class GAT_SIR(nn.Module):
    def __init__(self, g, input_size, hidden_size, output_size, gcn_nlayers, num_heads=5):
        super(GAT_SIR, self).__init__()
        self.g = g
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(dglnn.conv.GATConv(input_size, hidden_size, num_heads=num_heads,
                                                  residual=True, activation=F.relu))
        for i in range(gcn_nlayers):
            self.gcn_layers.append(dglnn.conv.GATConv(num_heads * hidden_size, hidden_size, num_heads=num_heads,
                                                      residual=True, activation=F.relu))

        self.linear_layers = nn.Sequential(
            nn.Linear(num_heads * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        h = x
        for layer in self.gcn_layers:
            h = layer(self.g, h).flatten(1)
        return self.linear_layers(h)

In [18]:
def sir_1d_output(params, I, D, target):
    gamma = params[:,0]
    beta = params[:,1]
    I[I != I] = 0.0
    D[D != D] = 0.0
    target[target != target] = 0.0
    err = th.tensor([0],dtype=th.float)
    err = err.to(device)
    S = pop - I - D

    I_new = th.add(th.add(I, th.mul(beta.view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(gamma.view(3142,1), I))

    return I_new

In [20]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 1000
batch_size = 1000
output_size = 2

device = th.device('cuda:0')
input_size = train_past_cases[0].shape[1] + attrs.shape[1]
net_sir = GAT_SIR(g, input_size, hidden_size, output_size, 1)
print('#features:', input_size)

# Move model to GPU
net_sir = net_sir.to(device)

# Move data to GPU
pop = pop.to(device)
attrs = attrs.to(device)
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_past_deaths[i] = train_past_deaths[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
    if len(train_labels_deaths[i].shape) == 1:
        train_labels_deaths[i] = train_labels_deaths[i].unsqueeze(1)
for i in range(len(valid_past_cases)):
    valid_past_cases[i] = valid_past_cases[i].to(device)
    valid_past_deaths[i] = valid_past_deaths[i].to(device)
    valid_labels_cases[i] = valid_labels_cases[i].to(device)
    if len(valid_labels_cases[i].shape) == 1:
         valid_labels_cases[i] = valid_labels_cases[i].unsqueeze(1)
    if len(test_labels_deaths[i].shape) == 1:
        valid_labels_deaths[i] = valid_labels_deaths[i].unsqueeze(1)

for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_past_deaths[i] = test_past_deaths[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
        test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)
    if len(test_labels_deaths[i].shape) == 1:
        test_labels_deaths[i] = test_labels_deaths[i].unsqueeze(1)
        
# Normalize attrs
attrs[th.isnan(attrs)] = 0
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_sir.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        th.manual_seed(1)
        vals = net_sir(batch)
        I = train_past_cases[idx][:,-1].view(3142,1)
        D = train_past_deaths[idx][:,-1].view(3142,1)
        # loss = sir_loss(vals, I, D, labels)
        output = sir_1d_output(vals, I, D, labels)
        loss = my_msle(output, labels)
        loss.backward(retain_graph=True)
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    count = 0
    with th.no_grad():
        eval_errs = []
        test_errs = []
        test_mses = []
        test_maes = []
        for idx in range(6):
            valid_feats = th.cat([valid_past_cases[idx], attrs], dim=1)
            eval_vals = net_sir(valid_feats)
            eval_I = valid_past_cases[idx][:,-1].view(3142,1)
            eval_D = valid_past_deaths[idx][:,-1].view(3142,1)
            eval_labels = valid_labels_cases[idx]
            # err = sir_loss(eval_vals, eval_I, eval_D, labels)
            eval_output = sir_1d_output(eval_vals, eval_I, eval_D, eval_labels)
            err = my_msle(eval_output, eval_labels)
            eval_errs.append(err.cpu().numpy())
            
            test_feats = th.cat([test_past_cases[idx], attrs], dim=1)
            test_vals = net_sir(test_feats)
            test_I = test_past_cases[idx][:,-1].view(3142,1)
            test_D = test_past_deaths[idx][:,-1].view(3142,1)
            test_labels = test_labels_cases[idx]
            
            test_output = sir_1d_output(test_vals, test_I, test_D, test_labels)
            test_err = th.mean((th.add(test_output,1).log() - th.add(test_labels_cases[idx],1).log())**2)
            test_mse = th.mean((test_output - test_labels_cases[idx])**2)
            test_mae = th.mean(th.abs(test_output - test_labels_cases[idx]))
            test_errs.append(test_err.cpu().numpy())
            test_mses.append(test_mse.cpu().numpy())
            test_maes.append(test_mae.cpu().numpy())
        print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs), np.mean(test_mses), np.mean(test_maes)))
        # print('epoch={}, loss={:.3f}, test loss={:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs)))

#features: 97


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


epoch=0, loss=0.064, validation loss = 0.124, test loss=nan, test mse = 1519538.875, test mae = 48.409
epoch=1, loss=0.026, validation loss = 0.021, test loss=nan, test mse = 1668.562, test mae = 7.821
epoch=2, loss=0.023, validation loss = 0.012, test loss=nan, test mse = 1736.254, test mae = 7.624
epoch=3, loss=0.021, validation loss = 0.010, test loss=nan, test mse = 2943.810, test mae = 7.917
epoch=4, loss=0.020, validation loss = 0.010, test loss=nan, test mse = 2637.126, test mae = 7.904
epoch=5, loss=0.020, validation loss = 0.010, test loss=nan, test mse = 1660.168, test mae = 7.553
epoch=6, loss=0.019, validation loss = 0.010, test loss=nan, test mse = 1659.189, test mae = 7.514
epoch=7, loss=0.019, validation loss = 0.009, test loss=nan, test mse = 1654.434, test mae = 7.448
epoch=8, loss=0.019, validation loss = 0.009, test loss=nan, test mse = 1652.884, test mae = 7.398
epoch=9, loss=0.018, validation loss = 0.009, test loss=nan, test mse = 1652.287, test mae = 7.363
epoch=

epoch=82, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.149
epoch=83, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.149
epoch=84, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.149
epoch=85, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.149
epoch=86, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.148
epoch=87, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.225, test mae = 7.148
epoch=88, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.148
epoch=89, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.148
epoch=90, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.148
epoch=91, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.22

epoch=163, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.254, test mae = 7.150
epoch=164, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.257, test mae = 7.151
epoch=165, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.258, test mae = 7.151
epoch=166, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.256, test mae = 7.151
epoch=167, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.254, test mae = 7.150
epoch=168, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.251, test mae = 7.150
epoch=169, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.250, test mae = 7.150
epoch=170, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.251, test mae = 7.150
epoch=171, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.253, test mae = 7.150
epoch=172, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.257, test ma

...

epoch=645, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.144
epoch=646, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.145
epoch=647, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.145
epoch=648, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.232, test mae = 7.147
epoch=649, loss=0.017, validation loss = 0.007, test loss=nan, test mse = 1651.261, test mae = 7.153
epoch=650, loss=0.020, validation loss = 0.007, test loss=0.007, test mse = 1651.243, test mae = 7.150
epoch=651, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.144
epoch=652, loss=0.018, validation loss = 0.007, test loss=0.005, test mse = 1651.226, test mae = 7.144
epoch=653, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=654, loss=0.019, validation loss = 0.007, test loss=0.005, test mse =

epoch=725, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=726, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=727, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=728, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=729, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=730, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=731, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=732, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=733, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=734, loss=0.019, validation loss = 0.007, test loss=0.005, test mse

epoch=805, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=806, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=807, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=808, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=809, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=810, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=811, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=812, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=813, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=814, loss=0.019, validation loss = 0.007, test loss=0.005, test mse

epoch=885, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=886, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=887, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=888, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=889, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=890, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=891, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=892, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=893, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=894, loss=0.019, validation loss = 0.007, test loss=0.005, test mse

epoch=965, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=966, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=967, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=968, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=969, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=970, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=971, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=972, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=973, loss=0.019, validation loss = 0.007, test loss=0.005, test mse = 1651.227, test mae = 7.143
epoch=974, loss=0.019, validation loss = 0.007, test loss=0.005, test mse

# 7-day Predictions

## Prepare Data

In [18]:
import numpy as np
import torch as th
attr_cols = []

# Set the variable to True if we want to specify features ourselves.
use_specified_fields = False

if use_specified_fields:
    f = open("fields.txt", "r")
    fields = f.readlines()
    for field in fields:
        field = field.rstrip()
        arr = np.array(df[field])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))
else:
    for i in range(6, 86):
        arr = np.array(df[keys[i]])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))

attrs = th.tensor(np.stack(attr_cols, 1), dtype=th.float32)
cases = []
for i in range(87, 216):
    cases.append(np.array(df[keys[i]]))
cases = np.stack(cases, 1)
deaths = []
for i in range(218, 347):
    deaths.append(np.array(df[keys[i]]))
deaths = np.stack(deaths, 1)

pop = attrs[:,5]
pop = pop.view(3142, 1)
pop[1834] = 1 

print('attrs:', attrs.shape)
print('cases:', cases.shape)
print('deaths:', deaths.shape)
print(attrs[0])
print(cases[0])
print(deaths[0])

attrs: torch.Size([3142, 78])
cases: (3142, 129)
deaths: (3142, 129)
tensor([ 4.0641e+01, -7.3955e+01,  4.0651e+01, -7.3955e+01,  1.0000e+00,
         2.5828e+06,  1.2550e+06,  1.3938e+06,  4.7379e-01,  3.5671e+05,
         3.5369e+04,  2.5047e+06,  3.4100e+01,  3.8667e+05,  3.0966e+05,
         6.3500e+02,  9.7000e+00,  1.9580e+02,  1.9500e+01,  1.3841e+01,
         2.3050e+01,  3.5442e+04,  9.3070e+03,  6.0000e+00,  1.2000e+01,
         3.1800e+02,  4.5415e+00,  9.0415e+04,  8.6783e+04,  8.1356e+04,
         7.8035e+04,  7.9604e+04,  7.6959e+04,  8.6716e+04,  8.3968e+04,
         9.6663e+04,  9.9134e+04,  1.0605e+05,  1.1679e+05,  9.7154e+04,
         1.0570e+05,  1.6196e+05,  1.7958e+05,  1.4949e+05,  1.7469e+05,
         6.3789e+04,  7.8116e+04,  5.5241e+04,  6.8866e+04,  6.4628e+04,
         8.8066e+04,  3.5780e+04,  5.8209e+04,  1.2524e+04,  2.8426e+04,
         1.5400e+02,  2.4000e+01,  3.3000e+01,  1.5100e+02,  3.4100e+02,
         4.9300e+02,  1.1160e+03,  2.1390e+03,  2.8520e

## MLP

In [195]:
import torch as th

num_days = cases.shape[1]
num_valid_days = 15
num_test_days = 15
train_valid_split = num_days - num_valid_days - num_test_days
valid_test_split = num_days - num_test_days
in_len = 20
out_len = 7


train_attr_idxs = []
train_past_cases = []
train_labels_cases = []
train_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    train_attr_idxs.append(np.arange(len(cases)))
    train_past_cases.append(cases[:,start:end_begin])
    train_labels_cases.append(cases[:, end_begin: end])
    train_labels_deaths.append(deaths[:, end_begin: end])


valid_attr_idxs = []
valid_past_cases = []
valid_labels_cases = []
valid_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1,
                    valid_test_split - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    valid_attr_idxs.append(np.arange(len(cases)))
    valid_past_cases.append(cases[:,start:end_begin])
    valid_labels_cases.append(cases[:,end_begin: end])
    valid_labels_deaths.append(deaths[:,end_begin: end])

    
test_attr_idxs = []
test_past_cases = []
test_labels_cases = []
test_labels_deaths = []
for start in range(valid_test_split - in_len - out_len + 1,
                    num_days - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    test_attr_idxs.append(np.arange(len(cases)))
    test_past_cases.append(cases[:,start:end_begin])
    test_labels_cases.append(cases[:,end_begin: end])
    test_labels_deaths.append(deaths[:,end_begin: end])

    
train_attr_idxs = np.concatenate(train_attr_idxs, axis=0)
train_past_cases = th.tensor(np.concatenate(train_past_cases, axis=0), dtype=th.float32)
train_labels_cases = th.tensor(np.concatenate(train_labels_cases, axis=0), dtype=th.float32)
train_labels_deaths = th.tensor(np.concatenate(train_labels_deaths, axis=0), dtype=th.float32)

valid_attr_idxs = np.concatenate(valid_attr_idxs, axis=0)
valid_past_cases = th.tensor(np.concatenate(valid_past_cases, axis=0), dtype=th.float32)
valid_labels_cases = th.tensor(np.concatenate(valid_labels_cases, axis=0), dtype=th.float32)
valid_labels_deaths = th.tensor(np.concatenate(valid_labels_deaths, axis=0), dtype=th.float32)

test_attr_idxs = np.concatenate(test_attr_idxs, axis=0)
test_past_cases = th.tensor(np.concatenate(test_past_cases, axis=0), dtype=th.float32)
test_labels_cases = th.tensor(np.concatenate(test_labels_cases, axis=0), dtype=th.float32)
test_labels_deaths = th.tensor(np.concatenate(test_labels_deaths, axis=0), dtype=th.float32)

print(train_attr_idxs.shape)
print(valid_attr_idxs.shape)
print(test_attr_idxs.shape)
print('train:', train_past_cases.shape)
print('valid:', valid_past_cases.shape)
print('test:', test_past_cases.shape)

(229366,)
(43988,)
(43988,)
train: torch.Size([229366, 19])
valid: torch.Size([43988, 19])
test: torch.Size([43988, 19])


In [26]:
hidden_size = 50
out_size = 7
learning_rate = 0.000001
num_epochs = 1000
batch_size = 1000

print('mean of training labels: {:.3f}, mean of testing labels: {:.3f}'.format(th.mean(train_labels_cases),
                                                                               th.mean(valid_labels_cases)))

device = th.device('cuda:0')
input_size = train_past_cases.shape[1] + attrs.shape[1]
net_mlp = Net(input_size, hidden_size, out_size)
print('#features:', input_size)

# Move model to GPU
net_mlp = net_mlp.to(device)

# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
train_past_cases = train_past_cases.to(device)
train_labels_cases = train_labels_cases.to(device)
train_labels_deaths = train_labels_deaths.to(device)
if len(train_labels_cases.shape) == 1:
    train_labels_cases = train_labels_cases.unsqueeze(1)
if len(train_labels_deaths.shape) == 1:
    train_labels_deaths = train_labels_deaths.unsqueeze(1)
    
valid_past_cases = valid_past_cases.to(device)
valid_labels_cases = valid_labels_cases.to(device)
valid_labels_deaths = valid_labels_deaths.to(device)
if len(valid_labels_cases.shape) == 1:
    valid_labels_cases = valid_labels_cases.unsqueeze(1)
if len(valid_labels_deaths.shape) == 1:
    valid_labels_deaths = valid_labels_deaths.unsqueeze(1)

test_past_cases = test_past_cases.to(device)
test_labels_cases = test_labels_cases.to(device)
test_labels_deaths = test_labels_deaths.to(device)
if len(test_labels_cases.shape) == 1:
    test_labels_cases = test_labels_cases.unsqueeze(1)
if len(test_labels_deaths.shape) == 1:
    test_labels_deaths = test_labels_deaths.unsqueeze(1)

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_mlp.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    sample_idxs = th.tensor(range(len(train_past_cases)))
    batch_idxs = th.split(sample_idxs, batch_size)
    losses = []
    for idx in batch_idxs:
        attr_idx = train_attr_idxs[idx]
        labels = train_labels_cases[idx]
        labels[th.isnan(labels)] = 0
        batch = th.cat([train_past_cases[idx], attrs[attr_idx]], dim=1)
        vals = net_mlp(batch)
        
        loss = my_msle(vals, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        attr_idx = valid_attr_idxs
        valid_feats = th.cat([valid_past_cases, attrs[attr_idx]], dim=1)
        eval_vals = net_mlp(valid_feats)
        eval_err = my_msle(eval_vals, valid_labels_cases)
        
        test_feats = th.cat([test_past_cases, attrs[test_attr_idxs]], dim=1)
        test_vals = net_mlp(test_feats) 
        test_errs = my_msle(test_vals, test_labels_cases)
        test_mse = th.mean((test_vals - test_labels_cases).pow(2))
        test_mae = th.mean(th.abs(test_vals - test_labels_cases))
    print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), eval_err, test_errs, test_mse, test_mae))
    

mean of training labels: 66.839, mean of testing labels: 357.464
#features: 97
epoch=0, loss=3.486, validation loss = 12.291, test loss=14.244, test mse = 7826966.000, test mae = 467.574
epoch=1, loss=3.413, validation loss = 11.967, test loss=13.824, test mse = 7809968.000, test mae = 467.093
epoch=2, loss=3.365, validation loss = 11.779, test loss=13.594, test mse = 7794143.000, test mae = 466.643
epoch=3, loss=3.316, validation loss = 11.615, test loss=13.411, test mse = 7778914.000, test mae = 466.208
epoch=4, loss=3.254, validation loss = 11.244, test loss=12.989, test mse = 7762647.500, test mae = 465.732
epoch=5, loss=3.201, validation loss = 10.885, test loss=12.458, test mse = 7738763.000, test mae = 465.060
epoch=6, loss=3.160, validation loss = 10.656, test loss=12.164, test mse = 7716456.500, test mae = 464.422
epoch=7, loss=3.125, validation loss = 10.473, test loss=11.936, test mse = 7694360.000, test mae = 463.789
epoch=8, loss=3.094, validation loss = 10.314, test loss=

epoch=76, loss=1.753, validation loss = 4.857, test loss=5.505, test mse = 4659529.500, test mae = 358.814
epoch=77, loss=1.740, validation loss = 4.649, test loss=5.114, test mse = 4587366.500, test mae = 355.679
epoch=78, loss=1.663, validation loss = 4.192, test loss=4.597, test mse = 4504448.000, test mae = 352.182
epoch=79, loss=1.609, validation loss = 3.961, test loss=4.326, test mse = 4420351.500, test mae = 348.602
epoch=80, loss=1.576, validation loss = 3.815, test loss=4.158, test mse = 4338863.000, test mae = 345.055
epoch=81, loss=1.552, validation loss = 3.702, test loss=4.030, test mse = 4258905.000, test mae = 341.510
epoch=82, loss=1.531, validation loss = 3.608, test loss=3.925, test mse = 4180032.250, test mae = 337.952
epoch=83, loss=1.513, validation loss = 3.527, test loss=3.835, test mse = 4102009.250, test mae = 334.372
epoch=84, loss=1.497, validation loss = 3.454, test loss=3.755, test mse = 4024700.000, test mae = 330.765
epoch=85, loss=1.483, validation loss

epoch=153, loss=0.978, validation loss = 2.185, test loss=2.548, test mse = 3084761.250, test mae = 227.512
epoch=154, loss=0.972, validation loss = 2.185, test loss=2.551, test mse = 3222362.750, test mae = 232.855
epoch=155, loss=0.966, validation loss = 2.186, test loss=2.554, test mse = 3367699.250, test mae = 238.324
epoch=156, loss=0.960, validation loss = 2.186, test loss=2.557, test mse = 3520931.500, test mae = 244.062
epoch=157, loss=0.954, validation loss = 2.187, test loss=2.561, test mse = 3682231.500, test mae = 250.219
epoch=158, loss=0.948, validation loss = 2.188, test loss=2.565, test mse = 3851801.000, test mae = 256.762
epoch=159, loss=0.942, validation loss = 2.189, test loss=2.569, test mse = 4030828.750, test mae = 263.614
epoch=160, loss=0.937, validation loss = 2.191, test loss=2.573, test mse = 4219412.000, test mae = 270.765
epoch=161, loss=0.931, validation loss = 2.193, test loss=2.578, test mse = 4417303.000, test mae = 278.161
epoch=162, loss=0.926, valid

epoch=229, loss=0.769, validation loss = 2.769, test loss=3.379, test mse = 56939176.000, test mae = 1104.859
epoch=230, loss=0.770, validation loss = 2.783, test loss=3.396, test mse = 58593344.000, test mae = 1120.857
epoch=231, loss=0.771, validation loss = 2.796, test loss=3.413, test mse = 60281860.000, test mae = 1136.952
epoch=232, loss=0.773, validation loss = 2.810, test loss=3.430, test mse = 62005400.000, test mae = 1153.146
epoch=233, loss=0.774, validation loss = 2.824, test loss=3.447, test mse = 63764404.000, test mae = 1169.439
epoch=234, loss=0.775, validation loss = 2.837, test loss=3.464, test mse = 65559168.000, test mae = 1185.830
epoch=235, loss=0.777, validation loss = 2.851, test loss=3.481, test mse = 67389944.000, test mae = 1202.318
epoch=236, loss=0.778, validation loss = 2.865, test loss=3.499, test mse = 69257160.000, test mae = 1218.903
epoch=237, loss=0.779, validation loss = 2.880, test loss=3.516, test mse = 71161424.000, test mae = 1235.584
epoch=238,

epoch=304, loss=0.919, validation loss = 3.877, test loss=4.726, test mse = 294866496.000, test mae = 2509.642
epoch=305, loss=0.922, validation loss = 3.892, test loss=4.743, test mse = 299761440.000, test mae = 2530.239
epoch=306, loss=0.924, validation loss = 3.906, test loss=4.760, test mse = 304708832.000, test mae = 2550.884
epoch=307, loss=0.926, validation loss = 3.921, test loss=4.778, test mse = 309708480.000, test mae = 2571.573
epoch=308, loss=0.929, validation loss = 3.935, test loss=4.795, test mse = 314759808.000, test mae = 2592.302
epoch=309, loss=0.931, validation loss = 3.950, test loss=4.812, test mse = 319862016.000, test mae = 2613.067
epoch=310, loss=0.934, validation loss = 3.964, test loss=4.830, test mse = 325014336.000, test mae = 2633.864
epoch=311, loss=0.936, validation loss = 3.978, test loss=4.847, test mse = 330215520.000, test mae = 2654.688
epoch=312, loss=0.938, validation loss = 3.993, test loss=4.864, test mse = 335464992.000, test mae = 2675.534
e

...

epoch=675, loss=0.804, validation loss = 2.548, test loss=2.997, test mse = 53029116.000, test mae = 754.151
epoch=676, loss=0.803, validation loss = 2.542, test loss=2.987, test mse = 51219224.000, test mae = 735.545
epoch=677, loss=0.802, validation loss = 2.535, test loss=2.978, test mse = 49453836.000, test mae = 717.293
epoch=678, loss=0.802, validation loss = 2.530, test loss=2.969, test mse = 47732688.000, test mae = 699.725
epoch=679, loss=0.801, validation loss = 2.525, test loss=2.961, test mse = 46055824.000, test mae = 682.854
epoch=680, loss=0.801, validation loss = 2.520, test loss=2.953, test mse = 44423236.000, test mae = 666.931
epoch=681, loss=0.801, validation loss = 2.517, test loss=2.947, test mse = 42835088.000, test mae = 653.089
epoch=682, loss=0.800, validation loss = 2.514, test loss=2.941, test mse = 41291156.000, test mae = 639.986
epoch=683, loss=0.800, validation loss = 2.511, test loss=2.936, test mse = 39793572.000, test mae = 628.354
epoch=684, loss=0.8

epoch=751, loss=0.760, validation loss = 2.550, test loss=2.992, test mse = 8583053.000, test mae = 358.612
epoch=752, loss=0.758, validation loss = 2.537, test loss=2.975, test mse = 8504940.000, test mae = 356.434
epoch=753, loss=0.756, validation loss = 2.524, test loss=2.959, test mse = 8429941.000, test mae = 354.238
epoch=754, loss=0.755, validation loss = 2.512, test loss=2.943, test mse = 8358004.500, test mae = 352.022
epoch=755, loss=0.753, validation loss = 2.501, test loss=2.929, test mse = 8289296.500, test mae = 349.784
epoch=756, loss=0.751, validation loss = 2.489, test loss=2.914, test mse = 8223735.000, test mae = 347.527
epoch=757, loss=0.749, validation loss = 2.479, test loss=2.901, test mse = 8161204.000, test mae = 345.251
epoch=758, loss=0.747, validation loss = 2.468, test loss=2.888, test mse = 8101715.000, test mae = 342.958
epoch=759, loss=0.745, validation loss = 2.458, test loss=2.875, test mse = 8045368.000, test mae = 340.650
epoch=760, loss=0.744, valid

epoch=827, loss=0.695, validation loss = 2.246, test loss=2.638, test mse = 8757358.000, test mae = 395.264
epoch=828, loss=0.695, validation loss = 2.246, test loss=2.639, test mse = 8792312.000, test mae = 397.692
epoch=829, loss=0.694, validation loss = 2.247, test loss=2.640, test mse = 8824997.000, test mae = 400.021
epoch=830, loss=0.694, validation loss = 2.247, test loss=2.640, test mse = 8855362.000, test mae = 402.251
epoch=831, loss=0.694, validation loss = 2.247, test loss=2.641, test mse = 8883625.000, test mae = 404.389
epoch=832, loss=0.694, validation loss = 2.248, test loss=2.641, test mse = 8910155.000, test mae = 406.445
epoch=833, loss=0.694, validation loss = 2.248, test loss=2.642, test mse = 8935377.000, test mae = 408.433
epoch=834, loss=0.694, validation loss = 2.249, test loss=2.643, test mse = 8959503.000, test mae = 410.361
epoch=835, loss=0.694, validation loss = 2.249, test loss=2.643, test mse = 8982458.000, test mae = 412.228
epoch=836, loss=0.694, valid

epoch=903, loss=0.689, validation loss = 2.237, test loss=2.618, test mse = 6520458.500, test mae = 379.380
epoch=904, loss=0.689, validation loss = 2.236, test loss=2.617, test mse = 6452813.500, test mae = 377.213
epoch=905, loss=0.689, validation loss = 2.235, test loss=2.616, test mse = 6383913.500, test mae = 374.975
epoch=906, loss=0.689, validation loss = 2.234, test loss=2.615, test mse = 6313695.500, test mae = 372.658
epoch=907, loss=0.689, validation loss = 2.233, test loss=2.614, test mse = 6242051.500, test mae = 370.260
epoch=908, loss=0.688, validation loss = 2.233, test loss=2.612, test mse = 6169119.500, test mae = 367.781
epoch=909, loss=0.688, validation loss = 2.232, test loss=2.611, test mse = 6095254.000, test mae = 365.228
epoch=910, loss=0.688, validation loss = 2.231, test loss=2.610, test mse = 6020443.500, test mae = 362.602
epoch=911, loss=0.688, validation loss = 2.230, test loss=2.609, test mse = 5945090.500, test mae = 359.908
epoch=912, loss=0.688, valid

epoch=979, loss=0.685, validation loss = 2.301, test loss=2.729, test mse = 2935623.500, test mae = 233.286
epoch=980, loss=0.686, validation loss = 2.306, test loss=2.738, test mse = 2930309.500, test mae = 232.806
epoch=981, loss=0.686, validation loss = 2.311, test loss=2.747, test mse = 2925346.500, test mae = 232.334
epoch=982, loss=0.686, validation loss = 2.316, test loss=2.756, test mse = 2920687.750, test mae = 231.871
epoch=983, loss=0.686, validation loss = 2.321, test loss=2.765, test mse = 2916337.500, test mae = 231.421
epoch=984, loss=0.686, validation loss = 2.326, test loss=2.775, test mse = 2912280.750, test mae = 230.990
epoch=985, loss=0.687, validation loss = 2.332, test loss=2.785, test mse = 2908437.500, test mae = 230.571
epoch=986, loss=0.687, validation loss = 2.337, test loss=2.795, test mse = 2904651.750, test mae = 230.169
epoch=987, loss=0.687, validation loss = 2.343, test loss=2.806, test mse = 2900603.250, test mae = 229.782
epoch=988, loss=0.688, valid

## GAT

In [19]:
import torch as th

num_days = cases.shape[1]
num_valid_days = 15
num_test_days = 15
train_valid_split = num_days - num_valid_days - num_test_days
valid_test_split = num_days - num_test_days
in_len = 20
out_len = 7
out_size = 7

train_past_cases = []
train_past_deaths = []
train_labels_cases = []
train_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    train_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    train_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    train_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    train_labels_deaths.append(th.tensor(deaths[:,end_begin:end], dtype=th.float32))

valid_past_cases = []
valid_past_deaths = []
valid_labels_cases = []
valid_labels_deaths = []
for start in range(train_valid_split - in_len - out_len + 1, valid_test_split - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    valid_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    valid_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    valid_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    valid_labels_deaths.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    
test_past_cases = []
test_past_deaths = []
test_labels_cases = []
test_labels_deaths = []
for start in range(valid_test_split - in_len - out_len + 1, num_days - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    test_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    test_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    test_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    test_labels_deaths.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))

In [21]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 1000
batch_size = 1000
output_size = 7

device = th.device('cuda:0')
net_gat = GAT(g, train_past_cases[0].shape[1] + attrs.shape[1], hidden_size, output_size, 1)
# criterion = nn.MSELoss()
print('#features:', train_past_cases[0].shape[1] + attrs.shape[1])

# Move model to GPU
net_gat = net_gat.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
        
for i in range(len(valid_past_cases)):
    valid_past_cases[i] = valid_past_cases[i].to(device)
    valid_labels_cases[i] = valid_labels_cases[i].to(device)
    if len(valid_labels_cases[i].shape) == 1:
         valid_labels_cases[i] = valid_labels_cases[i].unsqueeze(1)

for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)            
            
# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_gat.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        #batch = train_past_cases[idx]
        vals = net_gat(batch)
        # loss = criterion(vals, labels)
        loss = my_msle(vals, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        eval_errs = []
        for cases, labels in zip(valid_past_cases, valid_labels_cases):
            valid_feats = th.cat([cases, attrs], dim=1)
            eval_vals = net_gat(valid_feats)
            err = my_msle(eval_vals, labels)
            eval_errs.append(err.cpu().numpy())
        
        test_errs = []
        test_mses = []
        test_maes = []
        for idx in range(6):
            test_feats = th.cat([test_past_cases[idx], attrs], dim=1)
            test_vals = net_gat(test_feats)
            test_labels = test_labels_cases[idx]
            test_err = my_msle(test_vals, test_labels)
            test_errs.append(test_err.cpu().numpy())
            test_mse = th.mean((test_vals - test_labels)**2)
            test_mses.append(test_mse.cpu().numpy())
            test_mae = th.mean(th.abs(test_vals - test_labels))
            test_maes.append(test_mae.cpu().numpy())
        print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs), np.mean(test_mses), np.mean(test_maes)))
        # print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs),np.mean(test_mses),np.mean(test_maes)))

#features: 97


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


epoch=0, loss=2.615, validation loss = 8.127, test loss=8.824, test mse = 6952212.000, test mae = 425.197
epoch=1, loss=2.309, validation loss = 6.974, test loss=7.496, test mse = 6910580.500, test mae = 422.623
epoch=2, loss=2.090, validation loss = 6.091, test loss=6.458, test mse = 6857885.500, test mae = 419.877
epoch=3, loss=1.933, validation loss = 5.436, test loss=5.753, test mse = 6805837.500, test mae = 417.239
epoch=4, loss=1.815, validation loss = 4.938, test loss=5.209, test mse = 6754297.500, test mae = 414.662
epoch=5, loss=1.722, validation loss = 4.548, test loss=4.778, test mse = 6697599.500, test mae = 412.185
epoch=6, loss=1.629, validation loss = 4.214, test loss=4.419, test mse = 6638038.500, test mae = 409.869
epoch=7, loss=1.539, validation loss = 3.916, test loss=4.097, test mse = 6569448.500, test mae = 407.461
epoch=8, loss=1.464, validation loss = 3.672, test loss=3.850, test mse = 6483552.000, test mae = 404.652
epoch=9, loss=1.406, validation loss = 3.485, 

epoch=77, loss=0.791, validation loss = 2.516, test loss=2.855, test mse = 3201492.000, test mae = 357.333
epoch=78, loss=0.788, validation loss = 2.511, test loss=2.848, test mse = 3179406.250, test mae = 356.683
epoch=79, loss=0.786, validation loss = 2.505, test loss=2.841, test mse = 3156165.750, test mae = 355.940
epoch=80, loss=0.783, validation loss = 2.498, test loss=2.834, test mse = 3132898.750, test mae = 355.262
epoch=81, loss=0.780, validation loss = 2.492, test loss=2.827, test mse = 3108451.000, test mae = 354.540
epoch=82, loss=0.778, validation loss = 2.485, test loss=2.819, test mse = 3082960.750, test mae = 353.657
epoch=83, loss=0.775, validation loss = 2.477, test loss=2.811, test mse = 3061773.000, test mae = 352.984
epoch=84, loss=0.772, validation loss = 2.470, test loss=2.803, test mse = 3039411.000, test mae = 352.078
epoch=85, loss=0.769, validation loss = 2.462, test loss=2.794, test mse = 3016403.750, test mae = 351.010
epoch=86, loss=0.766, validation loss

epoch=154, loss=0.501, validation loss = 1.277, test loss=1.414, test mse = 1496791.125, test mae = 230.370
epoch=155, loss=0.498, validation loss = 1.264, test loss=1.399, test mse = 1470135.000, test mae = 228.516
epoch=156, loss=0.494, validation loss = 1.251, test loss=1.384, test mse = 1442649.500, test mae = 226.571
epoch=157, loss=0.491, validation loss = 1.238, test loss=1.369, test mse = 1415660.375, test mae = 224.617
epoch=158, loss=0.487, validation loss = 1.226, test loss=1.354, test mse = 1388427.125, test mae = 222.646
epoch=159, loss=0.484, validation loss = 1.213, test loss=1.339, test mse = 1361358.125, test mae = 220.681
epoch=160, loss=0.480, validation loss = 1.200, test loss=1.324, test mse = 1336668.000, test mae = 218.734
epoch=161, loss=0.477, validation loss = 1.187, test loss=1.309, test mse = 1310007.375, test mae = 216.728
epoch=162, loss=0.474, validation loss = 1.175, test loss=1.295, test mse = 1282418.875, test mae = 214.715
epoch=163, loss=0.470, valid

...

epoch=688, loss=0.116, validation loss = 0.177, test loss=0.211, test mse = 3585107.250, test mae = 210.362
epoch=689, loss=0.116, validation loss = 0.177, test loss=0.210, test mse = 3538676.750, test mae = 208.960
epoch=690, loss=0.116, validation loss = 0.176, test loss=0.209, test mse = 3490841.250, test mae = 207.442
epoch=691, loss=0.116, validation loss = 0.175, test loss=0.208, test mse = 3445430.250, test mae = 206.080
epoch=692, loss=0.115, validation loss = 0.175, test loss=0.208, test mse = 3404771.750, test mae = 205.033
epoch=693, loss=0.115, validation loss = 0.175, test loss=0.208, test mse = 3367250.000, test mae = 204.187
epoch=694, loss=0.115, validation loss = 0.175, test loss=0.208, test mse = 3328985.250, test mae = 203.278
epoch=695, loss=0.115, validation loss = 0.174, test loss=0.207, test mse = 3286757.000, test mae = 202.066
epoch=696, loss=0.115, validation loss = 0.173, test loss=0.207, test mse = 3240628.750, test mae = 200.563
epoch=697, loss=0.115, valid

epoch=764, loss=0.107, validation loss = 0.146, test loss=0.170, test mse = 1239736.125, test mae = 130.415
epoch=765, loss=0.107, validation loss = 0.144, test loss=0.166, test mse = 1239123.375, test mae = 131.036
epoch=766, loss=0.107, validation loss = 0.147, test loss=0.168, test mse = 1228717.625, test mae = 130.928
epoch=767, loss=0.107, validation loss = 0.144, test loss=0.165, test mse = 1201384.375, test mae = 129.177
epoch=768, loss=0.106, validation loss = 0.144, test loss=0.167, test mse = 1162282.375, test mae = 126.476
epoch=769, loss=0.107, validation loss = 0.156, test loss=0.180, test mse = 1126436.875, test mae = 124.313
epoch=770, loss=0.107, validation loss = 0.154, test loss=0.178, test mse = 1109656.625, test mae = 123.441
epoch=771, loss=0.106, validation loss = 0.142, test loss=0.163, test mse = 1109819.000, test mae = 123.824
epoch=772, loss=0.106, validation loss = 0.144, test loss=0.164, test mse = 1110643.500, test mae = 124.590
epoch=773, loss=0.107, valid

epoch=841, loss=0.116, validation loss = 0.201, test loss=0.229, test mse = 227440.859, test mae = 61.493
epoch=842, loss=0.111, validation loss = 0.170, test loss=0.188, test mse = 210165.562, test mae = 58.711
epoch=843, loss=0.103, validation loss = 0.133, test loss=0.142, test mse = 187325.172, test mae = 55.996
epoch=844, loss=0.104, validation loss = 0.180, test loss=0.215, test mse = 167870.109, test mae = 56.791
epoch=845, loss=0.116, validation loss = 0.274, test loss=0.340, test mse = 157198.875, test mae = 58.388
epoch=846, loss=0.111, validation loss = 0.209, test loss=0.252, test mse = 154746.266, test mae = 56.558
epoch=847, loss=0.102, validation loss = 0.133, test loss=0.143, test mse = 159669.672, test mae = 53.680
epoch=848, loss=0.108, validation loss = 0.166, test loss=0.183, test mse = 167224.922, test mae = 54.292
epoch=849, loss=0.116, validation loss = 0.208, test loss=0.236, test mse = 170435.016, test mae = 55.792
epoch=850, loss=0.118, validation loss = 0.213

epoch=919, loss=0.127, validation loss = 0.265, test loss=0.311, test mse = 203451.984, test mae = 68.388
epoch=920, loss=0.121, validation loss = 0.232, test loss=0.269, test mse = 228647.562, test mae = 71.582
epoch=921, loss=0.108, validation loss = 0.174, test loss=0.196, test mse = 274821.875, test mae = 78.118
epoch=922, loss=0.102, validation loss = 0.180, test loss=0.226, test mse = 339504.656, test mae = 89.485
epoch=923, loss=0.117, validation loss = 0.360, test loss=0.496, test mse = 403915.531, test mae = 102.000
epoch=924, loss=0.131, validation loss = 0.454, test loss=0.626, test mse = 430520.031, test mae = 106.576
epoch=925, loss=0.116, validation loss = 0.289, test loss=0.392, test mse = 401332.719, test mae = 99.785
epoch=926, loss=0.102, validation loss = 0.164, test loss=0.193, test mse = 342632.281, test mae = 87.758
epoch=927, loss=0.111, validation loss = 0.210, test loss=0.241, test mse = 293406.719, test mae = 80.037
epoch=928, loss=0.124, validation loss = 0.2

epoch=996, loss=0.141, validation loss = 0.375, test loss=0.456, test mse = 799230.438, test mae = 130.065
epoch=997, loss=0.123, validation loss = 0.282, test loss=0.339, test mse = 885133.500, test mae = 134.397
epoch=998, loss=0.106, validation loss = 0.216, test loss=0.259, test mse = 1008924.938, test mae = 143.771
epoch=999, loss=0.115, validation loss = 0.360, test loss=0.486, test mse = 1149371.375, test mae = 158.867


## GAT - SIR (constant parameters)

In [20]:
def sir_output(params, I, D, target):
    gamma = params[:,0]
    beta = params[:,1]
    I[I != I] = 0.0
    D[D != D] = 0.0
    target[target != target] = 0.0
    err = th.tensor([0],dtype=th.float)
    err = err.to(device)
    S = pop - I - D
    I_lst = []
    # for i in range(0):
    for i in range(7):
        S_new = th.add(S, - th.div(th.mul(beta.view(3142,1), th.mul(S, I)), pop))
        I_new = th.add(th.add(I, th.mul(beta.view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(gamma.view(3142,1), I))
        D_new = th.add(D, th.mul(gamma.view(3142,1), I))
        I_lst.append(I_new)
        temp = (I_new.add(1).log() - target[:,i].view(3142,1).add(1).log()).pow(2)
        err = err + th.mean(temp)
        S = S_new
        I = I_new
        D = D_new
    return th.cat(I_lst, dim = 1)

In [30]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 1000
batch_size = 1000
output_size = 2

device = th.device('cuda:0')
input_size = train_past_cases[0].shape[1] + attrs.shape[1]
net_sir = GAT_SIR(g, input_size, hidden_size, output_size, 1)
print('#features:', input_size)

# Move model to GPU
net_sir = net_sir.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
pop = pop.to(device)
attrs = attrs.to(device)
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_past_deaths[i] = train_past_deaths[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
    if len(train_labels_deaths[i].shape) == 1:
        train_labels_deaths[i] = train_labels_deaths[i].unsqueeze(1)
        
for i in range(len(valid_past_cases)):
    valid_past_cases[i] = valid_past_cases[i].to(device)
    valid_past_deaths[i] = valid_past_deaths[i].to(device)
    valid_labels_cases[i] = valid_labels_cases[i].to(device)
    if len(valid_labels_cases[i].shape) == 1:
         valid_labels_cases[i] = valid_labels_cases[i].unsqueeze(1)
    if len(test_labels_deaths[i].shape) == 1:
        valid_labels_deaths[i] = valid_labels_deaths[i].unsqueeze(1)

for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_past_deaths[i] = test_past_deaths[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
        test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)
    if len(test_labels_deaths[i].shape) == 1:
        test_labels_deaths[i] = test_labels_deaths[i].unsqueeze(1)
        
# Normalize attrs
attrs[th.isnan(attrs)] = 0
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net_sir.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        th.manual_seed(1)
        vals = net_sir(batch)
        I = train_past_cases[idx][:,-1].view(3142,1)
        D = train_past_deaths[idx][:,-1].view(3142,1)
        output = sir_output(vals, I, D, labels)
        loss = my_msle(output,labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    count = 0
    with th.no_grad():
        eval_errs = []
        test_errs = []
        test_mses = []
        test_maes = []
        for idx in range(6):
            valid_feats = th.cat([valid_past_cases[idx], attrs], dim=1)
            eval_vals = net_sir(valid_feats)
            eval_I = valid_past_cases[idx][:,-1].view(3142,1)
            eval_D = valid_past_deaths[idx][:,-1].view(3142,1)
            labels = valid_labels_cases[idx]
            eval_output = sir_output(eval_vals, eval_I, eval_D, labels)
            err = my_msle(eval_output, labels)
            # err = sir_loss_const(eval_vals, eval_I, eval_D, labels)
            eval_errs.append(err.cpu().numpy())
            
            test_feats = th.cat([test_past_cases[idx], attrs], dim=1)
            test_vals = net_sir(test_feats)
            test_I = test_past_cases[idx][:,-1].view(3142,1)
            test_D = test_past_deaths[idx][:,-1].view(3142,1)
            test_labels = test_labels_cases[idx]
            # test_err = sir_loss_const(test_vals, test_I, test_D, test_labels)
            test_output = sir_output(test_vals, test_I, test_D, test_labels)
            test_err = my_msle(test_output, test_labels)
            test_mse = th.mean((test_output - test_labels_cases[idx])**2)
            test_mae = th.mean(th.abs(test_output - test_labels_cases[idx]))
            test_errs.append(test_err.cpu().numpy())
            test_mses.append(test_mse.cpu().numpy())
            test_maes.append(test_mae.cpu().numpy())
        print('epoch={}, loss={:.3f}, validation loss = {:.3f}, test loss={:.3f}, test mse = {:.3f}, test mae = {:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs), np.mean(test_errs), np.mean(test_mses), np.mean(test_maes)))
        # print('epoch={}, loss={:.3f}, test loss={:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs)))

#features: 97
epoch=0, loss=0.343, validation loss = 0.579, test loss=0.665, test mse = 1470221.375, test mae = 71.449
epoch=1, loss=0.191, validation loss = 0.184, test loss=0.123, test mse = 52966.625, test mae = 32.545
epoch=2, loss=0.166, validation loss = 0.141, test loss=0.073, test mse = 63605.781, test mae = 34.106
epoch=3, loss=0.157, validation loss = 0.128, test loss=0.065, test mse = 74789.680, test mae = 34.801
epoch=4, loss=0.147, validation loss = 0.114, test loss=0.057, test mse = 71122.516, test mae = 33.402
epoch=5, loss=0.140, validation loss = 0.101, test loss=0.051, test mse = 40573.789, test mae = 31.002
epoch=6, loss=0.135, validation loss = 0.094, test loss=0.047, test mse = 34238.770, test mae = 29.981
epoch=7, loss=0.131, validation loss = 0.089, test loss=0.044, test mse = 30359.900, test mae = 29.157
epoch=8, loss=0.128, validation loss = 0.084, test loss=0.042, test mse = 30229.125, test mae = 28.882
epoch=9, loss=0.125, validation loss = 0.080, test loss=0

epoch=79, loss=0.121, validation loss = 0.068, test loss=0.034, test mse = 30220.611, test mae = 28.423
epoch=80, loss=0.121, validation loss = 0.068, test loss=0.034, test mse = 30220.611, test mae = 28.423
epoch=81, loss=0.121, validation loss = 0.068, test loss=0.034, test mse = 30220.611, test mae = 28.424
epoch=82, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.611, test mae = 28.424
epoch=83, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.615, test mae = 28.424
epoch=84, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.617, test mae = 28.425
epoch=85, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.617, test mae = 28.425
epoch=86, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.617, test mae = 28.425
epoch=87, loss=0.120, validation loss = 0.068, test loss=0.034, test mse = 30220.619, test mae = 28.425
epoch=88, loss=0.120, validation loss = 0.068, test loss=0.034, 

epoch=158, loss=0.110, validation loss = 0.064, test loss=0.033, test mse = 30220.604, test mae = 28.421
epoch=159, loss=0.109, validation loss = 0.064, test loss=0.033, test mse = 30220.607, test mae = 28.419
epoch=160, loss=0.110, validation loss = 0.064, test loss=0.033, test mse = 30220.609, test mae = 28.418
epoch=161, loss=0.110, validation loss = 0.064, test loss=0.033, test mse = 30220.607, test mae = 28.418
epoch=162, loss=0.109, validation loss = 0.064, test loss=0.033, test mse = 30220.600, test mae = 28.419
epoch=163, loss=0.109, validation loss = 0.064, test loss=0.033, test mse = 30220.580, test mae = 28.421
epoch=164, loss=0.110, validation loss = 0.064, test loss=0.034, test mse = 30220.656, test mae = 28.426
epoch=165, loss=0.109, validation loss = 0.064, test loss=0.033, test mse = 30220.570, test mae = 28.421
epoch=166, loss=0.109, validation loss = 0.064, test loss=0.033, test mse = 30220.588, test mae = 28.419
epoch=167, loss=0.110, validation loss = 0.064, test lo

epoch=237, loss=0.126, validation loss = 0.066, test loss=0.033, test mse = 30220.611, test mae = 28.409
epoch=238, loss=0.125, validation loss = 0.066, test loss=0.033, test mse = 30220.604, test mae = 28.410
epoch=239, loss=0.125, validation loss = 0.066, test loss=0.033, test mse = 30220.594, test mae = 28.411
epoch=240, loss=0.124, validation loss = 0.066, test loss=0.033, test mse = 30220.578, test mae = 28.412
epoch=241, loss=0.123, validation loss = 0.066, test loss=0.033, test mse = 30220.553, test mae = 28.414
epoch=242, loss=0.122, validation loss = 0.066, test loss=0.033, test mse = 30220.518, test mae = 28.415
epoch=243, loss=0.121, validation loss = 0.066, test loss=0.033, test mse = 30220.455, test mae = 28.417
epoch=244, loss=0.120, validation loss = 0.066, test loss=0.033, test mse = 30220.352, test mae = 28.420
epoch=245, loss=0.120, validation loss = 0.066, test loss=0.033, test mse = 30220.178, test mae = 28.422
epoch=246, loss=0.120, validation loss = 0.067, test lo

epoch=316, loss=0.113, validation loss = 0.064, test loss=0.033, test mse = 30220.588, test mae = 28.408
epoch=317, loss=0.112, validation loss = 0.064, test loss=0.033, test mse = 30220.576, test mae = 28.409
epoch=318, loss=0.112, validation loss = 0.064, test loss=0.033, test mse = 30220.553, test mae = 28.410
epoch=319, loss=0.112, validation loss = 0.064, test loss=0.033, test mse = 30220.523, test mae = 28.410
epoch=320, loss=0.112, validation loss = 0.063, test loss=0.033, test mse = 30220.492, test mae = 28.411
epoch=321, loss=0.113, validation loss = 0.063, test loss=0.033, test mse = 30220.486, test mae = 28.411
epoch=322, loss=0.112, validation loss = 0.063, test loss=0.033, test mse = 30220.514, test mae = 28.411
epoch=323, loss=0.111, validation loss = 0.063, test loss=0.033, test mse = 30220.545, test mae = 28.411
epoch=324, loss=0.111, validation loss = 0.064, test loss=0.033, test mse = 30220.562, test mae = 28.410
epoch=325, loss=0.111, validation loss = 0.064, test lo

...

epoch=632, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=633, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=634, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=635, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=636, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=637, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.674, test mae = 28.401
epoch=638, loss=0.139, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=639, loss=0.138, validation loss = 0.067, test loss=0.033, test mse = 30220.672, test mae = 28.401
epoch=640, loss=0.138, validation loss = 0.067, test loss=0.033, test mse = 30220.674, test mae = 28.401
epoch=641, loss=0.138, validation loss = 0.067, test lo

epoch=711, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=712, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=713, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=714, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=715, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=716, loss=0.132, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=717, loss=0.131, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=718, loss=0.131, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=719, loss=0.131, validation loss = 0.068, test loss=0.033, test mse = 30220.678, test mae = 28.405
epoch=720, loss=0.131, validation loss = 0.068, test lo

epoch=790, loss=0.121, validation loss = 0.068, test loss=0.035, test mse = 30220.662, test mae = 28.428
epoch=791, loss=0.121, validation loss = 0.068, test loss=0.035, test mse = 30220.664, test mae = 28.429
epoch=792, loss=0.121, validation loss = 0.068, test loss=0.035, test mse = 30220.664, test mae = 28.429
epoch=793, loss=0.120, validation loss = 0.068, test loss=0.035, test mse = 30220.666, test mae = 28.430
epoch=794, loss=0.120, validation loss = 0.068, test loss=0.035, test mse = 30220.666, test mae = 28.430
epoch=795, loss=0.120, validation loss = 0.068, test loss=0.035, test mse = 30220.670, test mae = 28.430
epoch=796, loss=0.120, validation loss = 0.068, test loss=0.035, test mse = 30220.672, test mae = 28.431
epoch=797, loss=0.120, validation loss = 0.068, test loss=0.035, test mse = 30220.672, test mae = 28.431
epoch=798, loss=0.119, validation loss = 0.068, test loss=0.035, test mse = 30220.670, test mae = 28.431
epoch=799, loss=0.119, validation loss = 0.068, test lo

epoch=869, loss=0.111, validation loss = 0.063, test loss=0.033, test mse = 30220.303, test mae = 28.414
epoch=870, loss=0.111, validation loss = 0.063, test loss=0.033, test mse = 30220.266, test mae = 28.414
epoch=871, loss=0.111, validation loss = 0.063, test loss=0.033, test mse = 30220.232, test mae = 28.414
epoch=872, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30220.195, test mae = 28.413
epoch=873, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30220.150, test mae = 28.413
epoch=874, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30220.100, test mae = 28.413
epoch=875, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30219.998, test mae = 28.412
epoch=876, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30219.877, test mae = 28.411
epoch=877, loss=0.110, validation loss = 0.062, test loss=0.033, test mse = 30219.781, test mae = 28.411
epoch=878, loss=0.110, validation loss = 0.062, test lo

epoch=948, loss=0.105, validation loss = 0.062, test loss=0.031, test mse = 30219.006, test mae = 28.401
epoch=949, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.447, test mae = 28.407
epoch=950, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.693, test mae = 28.409
epoch=951, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.799, test mae = 28.409
epoch=952, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.812, test mae = 28.409
epoch=953, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.758, test mae = 28.409
epoch=954, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.611, test mae = 28.409
epoch=955, loss=0.105, validation loss = 0.062, test loss=0.032, test mse = 30219.303, test mae = 28.406
epoch=956, loss=0.105, validation loss = 0.062, test loss=0.031, test mse = 30219.117, test mae = 28.402
epoch=957, loss=0.105, validation loss = 0.062, test lo

In [29]:
test_output.shape

torch.Size([3142, 7])